In [5]:
#Celda para librerías
import sklearn as sk


import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import StratifiedKFold

# LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb



from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

#app_train_def = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v1.csv')
app_train_def_2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v2.csv')
app_train_def_3 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v3.csv')
print(app_train_def_3.info())

2.1.3
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 750 entries, TARGET to LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Sold
dtypes: bool(148), float64(560), int64(42)
memory usage: 1.4 GB
None


In [6]:
#Train y Test
X = app_train_def_3.drop(columns = ['TARGET'])
y = app_train_def_3['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [49]:
#Definimos espacio de hiperparámetros para LightGBM
space_lightGBM = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo

    'num_leaves': hp.quniform('num_leaves', 20, 40, 1), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),  
    'min_child_samples': hp.quniform('min_child_samples', 50, 90, 1),  
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.02),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.001, 0.1),
    'max_bin': hp.quniform('max_bin', 250, 350, 10),
    'feature_fraction': hp.uniform('feature_fraction', 0.2, 0.5),
    'max_depth': hp.choice('max_depth', [-1, 3, 5]), 
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 1, 5, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 110),  
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 0.2),  
    
    'scale_pos_weight': 1,  # Fijo
    'is_unbalance': False  # Fijo
}

Trials_lightGBM_Ligero = Trials()


In [50]:
n_iter = 400

def F_OPT(params):
    params['max_bin'] = int(params['max_bin'])
    params['num_leaves'] = int(params['num_leaves'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['min_child_samples'] = int(params['min_child_samples'])

    model = LGBMClassifier(
        n_estimators=n_iter,
        boosting_type=params['boosting_type'],
        objective=params['objective'],
        metric=params['metric'],
        num_leaves=params['num_leaves'],
        learning_rate=params['learning_rate'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        min_gain_to_split=params['min_gain_to_split'],
        max_bin=params['max_bin'],
        feature_fraction=params['feature_fraction'],
        max_depth=params['max_depth'],
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=params['bagging_freq'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        is_unbalance=params['is_unbalance'],
        random_state=42)
    
    cv_Strat = StratifiedKFold(n_splits=5)
    auc_lightGBM_Ligero = cross_val_score(model, X_train, y_train, cv=cv_Strat, scoring='roc_auc').mean()
    return {'loss': -auc_lightGBM_Ligero, 'status': STATUS_OK}
    

In [51]:
%%capture
best_lightGBM_Ligero = fmin(fn=F_OPT,
                            space=space_lightGBM,
                            algo=tpe.suggest,
                            max_evals=50,
                            trials=Trials_lightGBM_Ligero,
                            rstate= np.random.default_rng(42)
)

best_lightGBM_Ligero['num_leaves'] = int(best_lightGBM_Ligero['num_leaves'])
best_lightGBM_Ligero['max_bin'] = int(best_lightGBM_Ligero['max_bin'])
best_lightGBM_Ligero['min_child_samples'] = int(best_lightGBM_Ligero['min_child_samples'])
best_lightGBM_Ligero['bagging_freq'] = int(best_lightGBM_Ligero['bagging_freq'])
best_lightGBM_Ligero['min_child_samples'] = int(best_lightGBM_Ligero['min_child_samples'])

#print(best_lightGBM_Ligero)

In [52]:
import pickle
with open("Trials_LightGBM_Ligero_v3_server_50_evals.pkl", "wb") as f:
    pickle.dump(Trials_lightGBM_Ligero, f)
with open("Trials_lightGBM_Ligero_v3_server_50_evals.pkl", "rb") as f:
    Trials_lightGBM_Ligero = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'Trials_lightGBM_Ligero_v3_server_50_evals.pkl'

In [53]:
#Mostramos los resultados de los trials 
from tabulate import tabulate

def trials_to_dataframe(trials):
    """
    Convierte la info de 'trials' de Hyperopt en un DataFrame 
    con columnas relevantes (loss, hiperparámetros, etc.).
    """
    rows = []
    for trial in trials.trials:
        # trial['result']['loss'] -> la métrica
        # trial['misc']['vals'] -> diccionario de hiperparámetros propuestos
        loss = trial['result']['loss']
        vals = trial['misc']['vals']
        
        # Convertir vals a algo "plano"
        row = {**vals}
        row['loss'] = loss
        rows.append(row)
    
    # df con columnas = keys (vals + 'loss')
    df = pd.DataFrame(rows)
    
    # A veces los hps que eran hp.quniform se quedan en listas => df['num_leaves'].apply(lambda x: x[0] if isinstance(x, list) else x)
    for c in df.columns:
        df[c] = df[c].apply(lambda x: x[0] if isinstance(x, list) else x)
    
    return df

# Para usarlo:
results_df = trials_to_dataframe(Trials_lightGBM_Ligero)
# Ordenamos por 'loss' asc => AUC mayor es 'loss' menor
results_df_sorted = results_df.sort_values(by='loss', ascending=True)
best_10 = results_df_sorted.head(10)
print(tabulate(best_10, headers='keys', tablefmt='psql'))

+----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------+
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 24 |           0.640067 |              5 |           0.282499 |       0.0297273 |       250 |           0 |                  85 |         0.0135347  |          0.0491041  |           39 |   0.0778152 |     18.3457  | -0.774915 |
| 21 |           0.63657  |              4 |           0.291883 |       0.02

In [54]:
#Vamos a realizar una selección de variables previa a una optimización más completa y una búsqueda SS
# más exhaustiva de hiperparámetros. Para ello, vamos a utilizar Permutation Importance.
from sklearn.inspection import permutation_importance

params_ligero = best_lightGBM_Ligero.copy()

model_ligero = LGBMClassifier(
    n_estimators=n_iter,          # = 200 (tu fase “ligera”)
    boosting_type='gbdt',         # Fijo
    objective='binary',           # Fijo
    metric='auc',                 # Fijo
    # Luego pasas los HP que has encontrado en la búsqueda
    num_leaves=params_ligero['num_leaves'],
    learning_rate=params_ligero['learning_rate'],
    min_child_samples=params_ligero['min_child_samples'],
    min_child_weight=params_ligero['min_child_weight'],
    min_gain_to_split=params_ligero['min_gain_to_split'],
    max_bin=params_ligero['max_bin'],
    feature_fraction=params_ligero['feature_fraction'],
    max_depth=params_ligero['max_depth'],
    bagging_fraction=params_ligero['bagging_fraction'],
    bagging_freq=params_ligero['bagging_freq'],
    reg_lambda=params_ligero['reg_lambda'],
    reg_alpha=params_ligero['reg_alpha'],
    scale_pos_weight=1,           # Fijo
    is_unbalance=False,           # Fijo
    random_state=42
)

model_ligero.fit(X_train, y_train)


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_gain_to_split is set=0.04910407918700191, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.04910407918700191
[LightGBM] [Warning] feature_fraction is set=0.2824989392056802, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2824989392056802
[LightGBM] [Warning] bagging_fraction is set=0.6400674501889978, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6400674501889978
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_gain_to_split is set=0.04910407918700191, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.04910407918700191
[LightGBM] [Warning] feature_fraction is set=0.2824989392056802, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2824989392056802
[LightGBM] [Warni

LGBMClassifier(bagging_fraction=np.float64(0.6400674501889978), bagging_freq=5,
               feature_fraction=np.float64(0.2824989392056802),
               is_unbalance=False,
               learning_rate=np.float64(0.029727319064918552), max_bin=250,
               max_depth=np.int64(0), metric='auc', min_child_samples=85,
               min_child_weight=np.float64(0.01353468070382758),
               min_gain_to_split=np.float64(0.04910407918700191),
               n_estimators=400, num_leaves=39, objective='binary',
               random_state=42, reg_alpha=np.float64(0.07781524410752336),
               reg_lambda=np.float64(18.3456623180481), scale_pos_weight=1)

In [ ]:
%%capture
pi = permutation_importance(
    estimator=model_ligero,
    X=X_train,
    y=y_train,
    scoring='roc_auc',
    n_repeats=20,
    random_state=42
)

In [57]:
#with open("pi_lightGBM_Ligero_v3_server_20_reapeats.pkl", "wb") as f:
#    pickle.dump(pi, f)

with open("/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/pi_lightGBM_Ligero_v3_server_20_reapeats.pkl", "rb") as f:
    pi = pickle.load(f)


In [ ]:

features = X_train.columns
importances_mean = pi.importances_mean
importances_std = pi.importances_std

#Lo unimos en un DF

df_pi = pd.DataFrame({
    'feature': features,
    'importance_mean': importances_mean,
    'importance_std': importances_std
})
df_pi.to_csv("Permutation_Importance_LightGBM_Ligero_v3_server_20_repeats.csv", index=False)

df_pi_sorted = df_pi.sort_values(by='importance_mean', ascending=False)
print(tabulate(df_pi_sorted, headers='keys', tablefmt='psql'))

In [ ]:
subset_features_pi = df_pi_sorted[df_pi_sorted['importance_mean'] > 0]
subset_features= subset_features_pi['feature'].tolist()
X_train_subset = X_train[subset_features]
y_train_subset = y_train
X_test_subset = X_test[subset_features]
y_test_subset = y_test
print(X_train_subset.describe())

In [8]:
space_lightGBM_completo = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo
    #Ajustamos los HP para centrarlos y reducir el rango a los resultados obtenidos anteriormente
    'num_leaves': hp.quniform('num_leaves', 20, 40, 1), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),  
    'min_child_samples': hp.quniform('min_child_samples', 50, 90, 1),  
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 0.01),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0.01, 0.1),
    'max_bin': hp.quniform('max_bin', 240, 300, 5),
    'feature_fraction': hp.uniform('feature_fraction', 0.2, 0.4),
    'max_depth': hp.choice('max_depth', [-1, 3]), 
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 1, 10, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 80),  
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 0.2),  
    
    'scale_pos_weight': 1,  # Fijo
    'is_unbalance': False  # Fijo
}

Trials_lightGBM_Completo = Trials()

In [9]:
n_iter = 2000
#Vamos a probar ahora a realizar esa búsqueda más extensa pero con seleccionando las variables que parecen tener mayor releveancia
def F_OPT_Completa(params):
    params['max_bin'] = int(params['max_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['bagging_freq'] = int(params['bagging_freq'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['num_leaves'] = int(params['num_leaves'])

    model = LGBMClassifier(
        n_estimators=n_iter,
        boosting_type=params['boosting_type'],
        objective=params['objective'],
        metric=params['metric'],
        num_leaves=params['num_leaves'],
        learning_rate=params['learning_rate'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        min_gain_to_split=params['min_gain_to_split'],
        max_bin=params['max_bin'],
        feature_fraction=params['feature_fraction'],
        max_depth=params['max_depth'],
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=params['bagging_freq'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        scale_pos_weight=params['scale_pos_weight'],
        is_unbalance=params['is_unbalance'],
        random_state=42
        )
    
    cv_Strat = StratifiedKFold(n_splits=5)
    auc_lightGBM_Completo = cross_val_score(model, X_train, y_train, cv=cv_Strat, scoring='roc_auc').mean()
    return {'loss': -auc_lightGBM_Completo, 'status': STATUS_OK}
    

In [11]:
%%capture
best_lightGBM_Completo = fmin(fn=F_OPT_Completa,
                            space=space_lightGBM_completo,
                            algo=tpe.suggest,
                            max_evals=100,
                            trials=Trials_lightGBM_Completo,
                            rstate= np.random.default_rng(42)
)
best_lightGBM_Completo['max_bin'] = int(best_lightGBM_Completo['max_bin'])
best_lightGBM_Completo['min_child_samples'] = int(best_lightGBM_Completo['min_child_samples'])
best_lightGBM_Completo['bagging_freq'] = int(best_lightGBM_Completo['bagging_freq'])
best_lightGBM_Completo['min_child_samples'] = int(best_lightGBM_Completo['min_child_samples'])
best_lightGBM_Completo['num_leaves'] = int(best_lightGBM_Completo['num_leaves'])


In [14]:
import pickle
#with open("Trials_LightGBM_Completo_v3_server_100_evals_csv_completo.pkl", "wb") as f:
#    pickle.dump(Trials_lightGBM_Completo, f)
with open("/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/Trials_LightGBM_Completo_v3_server_100_evals_csv_completo.pkl", "rb") as f:
    Trials_lightGBM_Completo = pickle.load(f)

In [15]:
#Mostramos los resultados de los trials 
from tabulate import tabulate

def trials_to_dataframe(trials):
    rows = []
    for trial in trials.trials:
        if 'loss' not in trial['result']:
            # saltar este trial
            continue

        loss = trial['result']['loss']
        vals = trial['misc']['vals']
        
        row = {**vals}
        row['loss'] = loss
        rows.append(row)
    
    df = pd.DataFrame(rows)

    for c in df.columns:
        df[c] = df[c].apply(lambda x: x[0] if isinstance(x, list) else x)

    return df


# Para usarlo:
results_df = trials_to_dataframe(Trials_lightGBM_Completo)
# Ordenamos por 'loss' asc => AUC mayor es 'loss' menor
results_df_sorted = results_df.sort_values(by='loss', ascending=True)
best_10 = results_df_sorted.head(10)
print(tabulate(best_10, headers='keys', tablefmt='psql'))

+----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------+
|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 65 |           0.946932 |             10 |           0.202777 |       0.0172655 |       250 |           0 |                  86 |         0.00728436 |           0.0451036 |           33 |   0.0558485 |      35.8496 | -0.779158 |
| 70 |           0.918794 |             10 |           0.217858 |       0.01

In [17]:
best_params = {
    'boosting_type': 'gbdt',  # Fijo
    'objective': 'binary',   # Fijo
    'metric': 'auc',         # Fijo
    'num_leaves': 33,        # Fijo
    'learning_rate': 0.0172655,   # Fijo
    'max_depth': -1,         # Fijo
    'bagging_fraction': 0.946932 ,
    'bagging_freq': 10,
    'feature_fraction':  0.202777 ,
    'max_bin': 250,
    'min_child_samples': 86,
    'min_child_weight': 0.00728436,
    'min_gain_to_split':  0.0451036,
    'reg_alpha': 0.0558485,
    'reg_lambda': 35.8496,
    'scale_pos_weight': 1,   # Fijo
    'is_unbalance': False,   # Fijo
}

final_model_lgb = LGBMClassifier(
    n_estimators=2000,
    boosting_type=best_params['boosting_type'],
    objective=best_params['objective'],
    metric=best_params['metric'],
    num_leaves=best_params['num_leaves'],
    learning_rate=best_params['learning_rate'],
    max_bin=best_params['max_bin'],
    max_depth=best_params['max_depth'],
    bagging_fraction=best_params['bagging_fraction'],
    bagging_freq=best_params['bagging_freq'],   
    feature_fraction=best_params['feature_fraction'],
    min_child_samples=best_params['min_child_samples'],
    min_child_weight=best_params['min_child_weight'],
    min_gain_to_split=best_params['min_gain_to_split'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    scale_pos_weight=1,
    is_unbalance=False,
    random_state=42
)

final_model_lgb.fit(X_train, y_train)

[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] min_gain_to_split is set=0.0451036, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0451036
[LightGBM] [Warning] feature_fraction is set=0.202777, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.202777
[LightGBM] [Warning] bagging_fraction is set=0.946932, subsample=1.0 will be ignored. Current value: bagging_fraction=0.946932
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] min_gain_to_split is set=0.0451036, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0451036
[LightGBM] [Warning] feature_fraction is set=0.202777, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.202777
[LightGBM] [Warning] bagging_fraction is set=0.946932, subsample=1.0 will be ignored. Current value: bagging_frac

LGBMClassifier(bagging_fraction=0.946932, bagging_freq=10,
               feature_fraction=0.202777, is_unbalance=False,
               learning_rate=0.0172655, max_bin=250, metric='auc',
               min_child_samples=86, min_child_weight=0.00728436,
               min_gain_to_split=0.0451036, n_estimators=2000, num_leaves=33,
               objective='binary', random_state=42, reg_alpha=0.0558485,
               reg_lambda=35.8496, scale_pos_weight=1)

In [20]:
y_pred_proba = final_model_lgb.predict_proba(X_test)[:, 1]
from sklearn.metrics import roc_auc_score
auc_test = roc_auc_score(y_test, y_pred_proba)
print(f'AUC en test: {auc_test}')

[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] min_gain_to_split is set=0.0451036, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0451036
[LightGBM] [Warning] feature_fraction is set=0.202777, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.202777
[LightGBM] [Warning] bagging_fraction is set=0.946932, subsample=1.0 will be ignored. Current value: bagging_fraction=0.946932
AUC en test: 0.7813628078996644


-AUC solo con app_train:
0.7687757918575168

-AUC añadiendo bureau.csv con threshhold de 0.00015:
0.7723138876162285

-AUC con threshold de 0.0001 y 40 features:
0.7741547798044517

-AUC con threshold de 0 y ~80 features:
0.777795871108371

Comienzo de pruebas en server
-Realizando Trials con 50 evaluaciones con todo el csv al completo el mejor resultado es de:

| 36 |           0.954169 |              4 |           0.316205 |       0.0289061 |       260 |           0 |                  62 |         0.00154331 |           0.0661147 |           39 |  0.134166   |      58.288  | -0.775321 |

Si probamos esta misma configuración sobre el test, es decir, HP con 50 evals y todos los features obtenemos: 
-AUC en test: 0.7807744950085171

-Realizando Trials con  100 evaluaciones con un rango más acotado de HP acorde a las pruebas anteriores, además de realizando una selección de variables con un threshold >0 (443 features) obtenemos:

|    |   bagging_fraction |   bagging_freq |   feature_fraction |   learning_rate |   max_bin |   max_depth |   min_child_samples |   min_child_weight |   min_gain_to_split |   num_leaves |   reg_alpha |   reg_lambda |      loss |
|----+--------------------+----------------+--------------------+-----------------+-----------+-------------+---------------------+--------------------+---------------------+--------------+-------------+--------------+-----------|
| 66 |           0.947598 |              5 |           0.256301 |       0.016433  |       270 |           0 |                  61 |         0.00531868 |           0.0774823 |           37 |   0.053589  |      37.9963 | -0.779018

Si probamos esta configuración sobre el test obtenemos:
-AUC en test: 0.7806464630531091 ---- Podemos observar que tenemos un resultado minimamente peor, pero puede deberse a fluctuaciones aleatorios, haber excluido alguena feature un poco útil... Aunque al ser una diferencia de ±0.0001–0.0002 de AUC se puede considerar que se encuentra dentro del margen de ruido.

-AUC en test: 0.7815232040819751 He puesto nstimators en 2000 en vez de 1000, que se me había pasado cambiarlo

Si añadimos los datos de la tabla bureau_balance y realizamos una optimización inicial (50), seleccionamos variables con threshold >0 (478 variables) y luego una optimizacón final (100), obtenemos un resultado de:
-AUC en test: 0.780960248215274

Si realizamos las pruebas sobre todo el csv sin realizar una selección de variables previa obtenemos:

| 65 |           0.946932 |             10 |           0.202777 |       0.0172655 |       250 |           0 |                  86 |         0.00728436 |           0.0451036 |           33 |   0.0558485 |      35.8496 | -0.779158 |

Y si realizamos las pruebas sobre el test:
-AUC en test: 0.7813628078996644 |